# Sleep Stage Scoring (on raw EEG signal)

What is different from [this version](https://github.com/baroquerock/sleep-stages-scoring):
1. Pytorch framework
2. Multichannel input
3. Different architecture

New architecture accomodates two channels:

<img src="img/arch2.png" width="800" height="1000">

In [ ]:
import random
import numpy as np
import pandas as pd
import os
import re
import copy
import time

import matplotlib.pyplot as plt
import torch
from torch import optim, nn
from torchvision import transforms, datasets, models

from collections import OrderedDict

from npzloader import DataLoader as DataLoader_x
import torch.utils.data as utils

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import (confusion_matrix, classification_report, cohen_kappa_score)

from hmmlearn.hmm import MultinomialHMM

from model_utils import print_hypnogram

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# loading fpz_cz and pz_oz channels
loader = DataLoader_x("../data/eeg_fpz_cz")
data_fp, labels = loader.load_data(verbose=0)
data_fp = {''.join(k.partition('fpz_cz')[1:]) : v for k, v in data_fp.items()} 

loader = DataLoader_x("../data/eeg_pz_oz")
data_pz, _ = loader.load_data(verbose=0)
data_pz = {''.join(k.partition('pz_oz')[1:]) : v for k, v in data_pz.items()} 

all_labels = np.hstack(labels.values())

In [ ]:
# Sleep stages values
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4

# int-to-label map
class_dict = {W: "W", 
              N1: "N1",
              N2: "N2",
              N3: "N3",
              REM: "REM"}

TEST_SUBJECT_INDEX = "09"
TEST_SUBJECT_FILE_PREFIX = f"SC4{TEST_SUBJECT_INDEX}"

### TRAINING
___

In [ ]:
"""
Function to randomly select subjects for partial cross-validation and prepare the data for training.
Train and test sets should be independent.
A patient with is SC413 is excluded from any test set because this patient has only one recording.
"""
def get_cv_data(data, labels, n_folds=3, ch='fpz_cz'):
    
    random.seed(29)
    test_key = []
    keys = list(labels.keys())
    keys = list(set([x[:5] for x in keys if '413' not in x]))
    for i in range(len(keys)):
        if f"4{TEST_SUBJECT_INDEX}" in keys[i]: 
            test_key = [keys[i]]
    random.shuffle(keys)
    print(test_key)
    val_keys = keys[:n_folds]
 
    cv_data = []
    test_key = [test_key[0] + '1E0',test_key[0] + '2E0']
    
    for fold in val_keys:
        fold_keys = [fold+'1E0', fold+'2E0']
        
        val_labels = [labels[key] for key in fold_keys]
        val_labels = np.hstack(val_labels)
        
        keys_not_training = fold_keys + test_key
        
        train_labels = [labels[key] for key in labels.keys() if key not in keys_not_training]
        train_labels = np.hstack(train_labels)
       
        test_labels=[labels[key] for key in test_key]
        test_labels= np.hstack(test_labels)
        
        if ch:
            val_data = [data['{}/{}'.format(ch,key)] for key in fold_keys]
            train_data = [data['{}/{}'.format(ch,key)] for key in labels.keys() if key not in keys_not_training]   
            test_data = [data['{}/{}'.format(ch,key)] for key in test_key]
        else:
            val_data = [data[key] for key in fold_keys]
            train_data = [data[key] for key in labels.keys() if key not in keys_not_training]
            test_data = [data[key] for key in labels.keys() if key in test_key]
           
        train_data = np.vstack(train_data)
        val_data = np.vstack(val_data)
        test_data = np.vstack(test_data)
    
        cv_data.append((train_data, train_labels, val_data, val_labels, test_data, test_labels))
    
    return cv_data 

"""
Function to slide on array with a moving window of size n
"""

def _slide(data, labels, n=3):
    
    #zero-indexing
    data = [data[i:i+n] for i, _ in enumerate(data[n:])]
    data = [np.vstack(x) for x in data]
    data = np.array(data)
    
    labels = labels[n:]
    
    assert data.shape[0] == labels.shape[0]
    
    return data, labels

"""
Function to do batch processing with _slide()
"""

def batch_slide(data_dict, labels_dict, n=3):
    
    keys = [(x,y) for x,y in zip(data_dict, labels_dict)]
    data = [_slide(data_dict[x], labels_dict[y], n=n) for x,y in keys]
    
    data_keys, label_keys = zip(*keys)
    data, labels = zip(*data)
    
    data_dict = {k:v for k,v in zip(data_keys, data)}
    data_dict = OrderedDict(sorted(data_dict.items()))
    
    labels_dict = {k:v for k,v in zip(label_keys, labels)}
    labels_dict = OrderedDict(sorted(labels_dict.items()))
        
    return data_dict, labels_dict 
  
  
"""
Function to merge channels into single recording.
axis = 0: merge channels along the first dimension - (3000,1) and (3000,1) --> (6000,1)
axis = 0: merge channels along the second dimension - (3000,1) and (3000,1) --> (3000,2)
"""  
    
def merge_channels(channels, axis=0):
    
    # number of records should be the same in all channels
    assert len(set([len(r) for r in channels])) == 1
    
    keys = [ch.keys() for ch in channels]
    
    merged = {}
    
    for records in zip(*keys):
        
        # assert that all recordings are for the same patient
        assert len(set([r.split(os.sep)[1] for r in records])) == 1
        vals = [ch[r] for r, ch in zip(records, channels)]
        if axis:
            stacked = np.dstack(vals)
        else:
            stacked = np.hstack(vals)
        patient = records[0].split(os.sep)[1]
        merged[patient] = stacked
        
    merged = OrderedDict(sorted(merged.items()))
    return merged


In [ ]:
data_all = merge_channels([data_fp, data_pz], axis=1)

In [ ]:
for key in ['SC4101E0', 'SC4102E0', 'SC4111E0', 'SC4112E0', 'SC4121E0', 'SC4122E0', 'SC4131E0', 'SC4141E0', 'SC4142E0', 'SC4151E0', 'SC4152E0', 'SC4161E0', 'SC4162E0', 'SC4171E0', 'SC4172E0', 'SC4181E0', 'SC4182E0', 'SC4191E0', 'SC4192E0']:
    del data_all[key]
    del labels[key]

In [ ]:
# getting cross-valivation data
# note: n_folds is not a number of folds in traditional sense
# the model is trained on 19 patients and validated on 1,
# so n_folds is a number of patients for validation
# to perform full cross-validation n_folds should be equal to 19 
# (not 20, because for SC413 patient there is only one record)
cv_data = get_cv_data(data_all, labels, ch=None, n_folds=1)

In [ ]:
def small_conv(fs): 
    return nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=fs//2, stride=fs//16, padding=2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.MaxPool1d(kernel_size=8, stride=8),
            
            nn.Conv1d(64, 64, kernel_size=8, padding=2),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=8, padding=2),
            nn.ReLU(), 
            nn.Conv1d(64, 64, kernel_size=8, padding=2),
            nn.ReLU())

def big_conv(fs): 
    return nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=fs*4, stride=fs//2, padding=2),
            nn.ReLU(), 
            nn.Dropout(0.3),
            nn.MaxPool1d(kernel_size=4, stride=4),
            
            nn.Conv1d(64, 64, kernel_size=6, padding=2),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=6, padding=2),
            nn.ReLU(), 
            nn.Conv1d(64, 64, kernel_size=6, padding=2),
            nn.ReLU())


class ConvNet(nn.Module):
  
    def __init__(self, n_cnn_dense=256, fs=100, num_classes=5):
      
        super(ConvNet, self).__init__()
        
        self.layer1_fz = small_conv(fs)        
        self.layer2_fz = big_conv(fs)

        self.layer1_pz = small_conv(fs)        
        self.layer2_pz = big_conv(fs)
        
        
        self.fc1 = nn.Sequential(
            nn.Linear(124, n_cnn_dense),
            nn.ReLU(),            
            nn.MaxPool1d(kernel_size=4, stride=4))
        
        self.fc2 = nn.Sequential(
            nn.Linear(4096, num_classes),
            nn.LogSoftmax(dim=1))
        
                
        
    def forward(self, channels):
      
        ch1 = channels[:, 0, :].unsqueeze(1)
        ch2 = channels[:, 1, :].unsqueeze(1)
        
        out1_1 = self.layer1_fz(ch1)
        out2_1 = self.layer2_fz(ch1)
        
        out1_2 = self.layer1_pz(ch2)
        out2_2 = self.layer2_pz(ch2)        
        
        out = torch.cat((out1_1, out2_1, out1_2, out2_2), dim=2)
        out = self.fc1(out)
        s = out.size()[0]
        out = out.view(s, -1)
        out = self.fc2(out)
       
        return out

In [ ]:
model = ConvNet()
train_data, train_labels, val_data, val_labels, test_data, test_labels = cv_data[0]

train = utils.TensorDataset(torch.from_numpy(np.swapaxes(train_data,1,2)), torch.from_numpy(train_labels).long())
train_loader = utils.DataLoader(train, batch_size=64, shuffle=True)

valid = utils.TensorDataset(torch.from_numpy(np.swapaxes(val_data,1,2)), torch.from_numpy(val_labels).long())
valid_loader = utils.DataLoader(valid, batch_size=64, shuffle=True)

test = utils.TensorDataset(torch.from_numpy(np.swapaxes(test_data,1,2)), torch.from_numpy(test_labels).long())
test_loader = utils.DataLoader(test, batch_size=64, shuffle=True)
                           
dataloaders = {'train': train_loader, 'valid': valid_loader, "test": test_loader}
dataset_sizes = {'train': len(train), 'valid': len(valid), "test": len(test)}

In [ ]:
def train_model(model, criteria, optimizer, scheduler, num_epochs=25, device='cuda'):
    model.to(device)
    since = time.time()
    epoch_loss_array=[] 
    accuracy_array=[]

    n_stages = len(class_dict.keys())
    emission_matrices = [np.zeros((n_stages,n_stages)) for _ in range(num_epochs)]
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0

    for epoch_idx, epoch in enumerate(range(num_epochs)):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'valid',"test"]:
            if phase == 'train':
                scheduler.step()
                model.train()  
            else:
                model.eval()   
                
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
   
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criteria(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                for y_pred, y_true in zip(preds, labels):
                    emission_matrices[epoch_idx][y_true, y_pred] += 1

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_loss_array.append(epoch_loss)
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            accuracy_array.append(epoch_acc)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'valid' and epoch_acc > best_acc:
                best_epoch = epoch_idx
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    # keep emission matrices according to the best model (biggest accuracy)
    print(f"Best epoch is {best_epoch}, with accuracy {best_acc:4f}")
    emission_matrix = emission_matrices[best_epoch]
    emission_matrix = emission_matrix / emission_matrix.sum(axis=1, keepdims=True)
    
    model.eval()
    
    return model, epoch_loss_array, accuracy_array, emission_matrix

In [ ]:
criteria = nn.NLLLoss() # negative log likelihood loss 
optimizer = optim.Adam(model.parameters(), lr=0.005)
sched = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)
nb_epochs = 1

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {}!".format(device))

model_ft,epoch_loss_array,accuracy_array,emission_matrix= train_model(model, criteria, optimizer, sched, nb_epochs, device)


In [ ]:
loss_array_train=[]
loss_array_valid=[]
loss_array_test=[]

for i in range(0, len(epoch_loss_array), 3):
    loss_array_train.append(epoch_loss_array[i])
    
    
for i in range(1, len(epoch_loss_array), 3):
    loss_array_valid.append(epoch_loss_array[i])

for i in range(2, len(epoch_loss_array), 3):
    loss_array_test.append(epoch_loss_array[i])
plt.plot(loss_array_train)



In [ ]:
plt.plot(loss_array_valid)

In [ ]:
plt.plot(loss_array_test)

In [ ]:
accuracy_array_train=[]
accuracy_array_valid=[]
accuracy_array_test=[]

for i in range(0, len(accuracy_array), 3):
    accuracy_array_train.append(accuracy_array[i])
    
for i in range(1,len(accuracy_array), 3):
    accuracy_array_valid.append(accuracy_array[i])

for i in range(2, len(accuracy_array), 3):
    accuracy_array_test.append(accuracy_array[i])
plt.plot(accuracy_array_train)


In [ ]:
plt.plot(accuracy_array_valid)

In [ ]:
plt.plot(accuracy_array_test)

## Testing on our subject nights
___

In [ ]:
# we want the predictions on the CPU, so that we can then display the results without busting GPU cache

# model_ft = model_ft.to("cpu")
model_ft_cnn = copy.deepcopy(model_ft)
model_ft_cnn_hmm = copy.deepcopy(model_ft)

In [ ]:
y_test_keys = [key for key in labels if TEST_SUBJECT_FILE_PREFIX in key]
y_test_keys

In [ ]:
plt.rcParams["figure.figsize"] = (20,5)

for night_key in y_test_keys:
    y_test_current = labels[night_key]
    curent_eeg_data = np.swapaxes(data_all[night_key], 1, 2) 
    current_eeg_tensor = torch.from_numpy(curent_eeg_data)
    
    output = model_ft_cnn.forward(current_eeg_tensor)
    _, y_test_current_pred = torch.max(output, 1)
    y_test_current_pred = y_test_current_pred.numpy()
    
    print(confusion_matrix(y_test_current, y_test_current_pred))

    print(classification_report(y_test_current, y_test_current_pred, target_names=class_dict.values()))

    print("Agreement score (Cohen Kappa): ", cohen_kappa_score(y_test_current, y_test_current_pred))
    
    print_hypnogram(
        [y_test_current, y_test_current_pred],
        labels=["scored", "predicted"],
        subject=TEST_SUBJECT_INDEX,
        night=night_key
    )

## Hidden Markov Model
___

We add a hidden markov model as a postprecessing step to take into account the data's normal transi

In [ ]:
y_train_valid_keys = [key for key in labels if TEST_SUBJECT_FILE_PREFIX not in key]


In [ ]:
plt.title("Emission table probabilities")
plt.imshow(emission_matrix)


In [ ]:
n_components = len(class_dict.keys())

def compute_hmm_matrices(y_train_valid_keys):
    transition_matrix = np.zeros((n_components,n_components))
    start_matrix = np.zeros((n_components))

    for night_key in y_train_valid_keys:
        print(f"Computing file: {night_key}")
        current_y = labels[night_key]
        start_matrix[current_y[0]] += 1

        for transition in zip(current_y[:-1], current_y[1:]):
            transition_matrix[transition[0], transition[1]] += 1
            
    transition_matrix = transition_matrix/transition_matrix.sum(axis=1, keepdims=True)
    start_matrix = start_matrix/start_matrix.sum()
    
    return transition_matrix, start_matrix
    
transition_matrix, start_matrix = compute_hmm_matrices(y_train_valid_keys)

In [ ]:
plt.title("Transition table probabilities")
plt.imshow(transition_matrix)

In [ ]:
hmm_model = MultinomialHMM(n_components=n_components)

hmm_model.transmat_ = transition_matrix
hmm_model.startprob_ = start_matrix
hmm_model.emissionprob_ = emission_matrix

print(hmm_model.transmat_,hmm_model.startprob_, hmm_model.emissionprob_)

In [ ]:
plt.rcParams["figure.figsize"] = (20,5)

for night_key in y_test_keys:
    y_test_current = labels[night_key]
    curent_eeg_data = np.swapaxes(data_all[night_key], 1, 2) 
    current_eeg_tensor = torch.from_numpy(curent_eeg_data)

    output = model_ft_cnn_hmm.forward(current_eeg_tensor)
    _, y_test_current_pred = torch.max(output, 1)
    y_test_current_pred = y_test_current_pred.numpy()
    print(y_test_current_pred)
    y_cnn_hmm_pred = hmm_model.predict(y_test_current_pred.reshape(-1, 1))
    
    print(confusion_matrix(y_test_current, y_cnn_hmm_pred))

    print(classification_report(y_test_current, y_cnn_hmm_pred, target_names=class_dict.values()))

    print("Agreement score (Cohen Kappa): ", cohen_kappa_score(y_test_current, y_cnn_hmm_pred))
    
    print_hypnogram(
        [y_test_current, y_cnn_hmm_pred],
        labels=["scored", "predicted"],
        subject=TEST_SUBJECT_INDEX,
        night=night_key
    )